In [ ]:
import numpy as np

def softmax(x):
  if (x.ndim == 1):
    x = x[None,:]    # ベクトル形状なら行列形状に変換
  # テンソル（x：行列）、軸（axis=1： 列の横方向に計算）
  return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# 入力（x）と出力（y）の例
x = np.array([[1,0,0], [0,1,0], [0,0,1]]) 
y = softmax(x)
print(y)  # 以下のように出力される
# [[0.57611688 0.21194156 0.21194156]  # → 猫： 全部足すと「1.0」になる
#  [0.21194156 0.57611688 0.21194156]  # → 虎： Σ＝1.0
#  [0.21194156 0.21194156 0.57611688]] # → ライオン： Σ＝1.0

In [1]:
!pip install mecab-python3

In [2]:
# encoding: utf-8
import MeCab
import glob
import os
import traceback
from bs4 import BeautifulSoup


class Sentence(object):
    def __init__(self, root):
        self.root = root
        self.surfaces = []
        self.features = []

        if self.root:
            node = root
            while node:
                self.surfaces.append(node.surface)
                self.features.append(node.feature)
                node = node.next

    def all_words(self):
        for surface, feature in zip(self.surfaces, self.features):
            yield surface, feature

    def word_count(self):
        return len(self.surfaces)

    def to_wakati(self):
        return ' '.join([w for w in self.surfaces if w])

class WakatiCorpus(object):
    def __init__(self):
        self.wakati_list = []

    def run(self):
        tagger = MeCab.Tagger('-Ochasen')
        tagger.parseToNode('') # to prevent GC
        
        for text in wiki_sentences():
            #print(text)
            encoded_text = text
            node = tagger.parseToNode(encoded_text)
            sentence = Sentence(node)
            wakati = sentence.to_wakati()
            if wakati:
                self.wakati_list.append(wakati + '\n')

    def load(self, filename):
        with open(filename, encoding='utf-8') as f:
            lines = f.readlines()
            self.wakati_list = [x.strip() for x in lines]

    def save(self, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            f.writelines(self.wakati_list)

    def loaded(self):
        return len(self.wakati_list) > 0


class DocumentIterator(object):

    WIKI_ROOT_DIR = './archive_wiki/jawiki_test'
    
    def __init__(self):
        self._i = 0
        self.document_path = os.path.join(DocumentIterator.WIKI_ROOT_DIR, '*')
        self._files = glob.glob(self.document_path)
        self._docs = []

    def __iter__(self):
        return self

    def __next__(self):
            if len(self._docs) > 0:
                doc = self._docs.pop()
                #print(self._docs[:5])
                return doc
            if self._i < len(self._files):
                i = self._i
                self._i += 1
                self._docs = self._get_doc_list(self._files[i])
                doc = self._docs.pop()
                return doc
            #print(self._docs[:5])
            if len(self._docs) == 0:
                raise StopIteration()


    def _get_doc_list(self, filename):
        try:
            with open(filename,encoding='utf-8') as f:
                xml = f.read()
                #print(xml[:5])
                soup = BeautifulSoup(xml, 'html.parser')
                #print(soup)
                docs = soup.find_all('doc')
                #print(docs[:5])
                #[print(doc.string[:5]) for doc in reversed(docs) if doc.string]
                return [doc.string for doc in reversed(docs) if doc.string]
        except:
            print('Failed to read', filename, traceback.format_exc())
            return []


class SentenceIterator(object):
    def __init__(self, document):
        self._i = 0
        self.sentences = self._break(document)

    def _break(self, sentences):
        lines = sentences.split('\n')
        return [line for line in lines if len(line) > 0]

    def __iter__(self):
        return self

    def __next__(self):
        if self._i < len(self.sentences):
            i = self._i
            self._i += 1
            return self.sentences[i]
        else:
            raise StopIteration()

def wiki_sentences():
    #print("YesYes")
    diter = DocumentIterator()
    #print(diter)
    for doc in diter:
        #print(doc[:5])
        siter = SentenceIterator(doc)
        for text in siter:
            yield text

if __name__ == '__main__':
    wakati = WakatiCorpus()
    wakati.run()
    wakati.save('archive_wiki/wakati/wakati_corpus.txt')
    #分かち書き文をファイル'archive_wiki/wakati/wakati_corpus.txt'に出力

In [3]:
import collections, os, pickle
import nltk, MeCab
import matplotlib.pyplot as plt

def tokenize(sentences):
    """文章を分かち書きするとともに、ボキャブラリも返す。

    :param sentences(str): 複数の文章を含む文字列。日本語想定。
    :return(list):
      tokens(list): 分かち書きした単語をlistとしてまとめたもの。
      vocab(list): ボキャブラリ。ユニークな単語一覧。
    """

    # 「。」、「！」、「？」で終わる一連の文字列を文として認識し分割する。
    #動詞、名詞、形容詞、形容動詞以外を分かち書き、語彙から削除、しかしこの語彙は使わない
    jp_sent_tokenizer = nltk.RegexpTokenizer(u'[^　「」！？。]*[！？。]')
    tagger = MeCab.Tagger('-Owakati -d mecab-ipadic-neologd')

    sents = jp_sent_tokenizer.tokenize(sentences)
    tokens = []
    vocab = []
    for sent in sents:
        node = tagger.parseToNode(sent)
        while node:
            features = node.feature.split(",")
            base_word = features[6]
            if base_word == "*" or base_word == " " or base_word == "\n":
                node = node.next
                continue
            if features[0] != "名詞"  and features[0] != "動詞" and features[0] != "形容詞" and features[0] != "形容動詞":
                node = node.next
                continue
            tokens.append(base_word)
            if base_word not in vocab:
                vocab.append(base_word)
            node = node.next
    return tokens, vocab

# 文章からボキャブラリと分かち書きを用意。
filename = "./archive_wiki/wakati/wakati_corpus.txt"
with open(filename, "rb") as fh:
    sentences = ""
    for line in fh.readlines():
        try:
            sentences += line.decode("utf-8") + " "
        except:
            continue
wakati_sentences, vocab = tokenize(sentences)
print("vocab[:5]", vocab[:5])
print("len(vocab)", len(vocab))

vocab[:5] ['意味', 'する', '記号', 'ラテン語', '合']
len(vocab) 120099


In [4]:
from collections import Counter
voc=Counter(wakati_sentences).most_common(20000)
#語彙データから出現頻度順20000語を抽出
    

In [5]:
len(voc)
new_voc=[voca[0] for voca in voc]
new_voc[:5]
#most_commonは出現回数も同時に出力するため単語のみを抽出

['する', 'れる', 'いる', '年', 'なる']

In [6]:
new_row[0][1]

NameError: name 'new_row' is not defined

In [7]:
from openpyxl import load_workbook

# Excelファイルのロード(読み取り専用)
excel_path='archive_wiki/D18/D18-2018.7.24.xlsx'
workbook = load_workbook(filename=excel_path, read_only=True)

# Excelのシート名一覧を表示
#print(workbook.sheetnames)
sheet=workbook[workbook.sheetnames[1]]
rows=[]
for row in sheet[f'A2:C{sheet.max_row}']:
    values = []
    for cell in row:
        values.append(cell.value)
    rows.append(values)
# ロードしたExcelファイルを閉じる
new_row=[[row[0],row[2]] for row in rows]

sheet=workbook[workbook.sheetnames[2]]
i=0
for row in sheet[f'C2:C{sheet.max_row}']:
    for cell in row:    
        if cell.value is not None :new_row[i][1]+=cell.value
    i=i+1
sheet=workbook[workbook.sheetnames[3]]
i=0
for row in sheet[f'C2:C{sheet.max_row}']:
    for cell in row:    
        if cell.value is not None :new_row[i][1]+=cell.value
    i=i+1

print(new_row[:5])
workbook.close()
#感性語辞典のエクセルの感性語と,その感情データ3人分を合体させて保存

[['哀', '悲悲悲'], ['愛', '好親安尊好親好'], ['相いれない', '不嫌嫌'], ['哀感', '悲悲寂悲'], ['愛敬', '親親好好親']]


In [8]:
wakati=MeCab.Tagger("-Owakati")
row_use=[]
for d in new_row:
    if d[0] == None or len(wakati.parse(d[0]).split())>1:
        continue
    else :row_use.append(d)
print(len(row_use))
row_use[:5]
#分かち書きをすると2語以上になる単語を削除。同時にエクセルファイルを読み込むときに発生した空のデータを削除

1401


[['哀', '悲悲悲'],
 ['愛', '好親安尊好親好'],
 ['相いれない', '不嫌嫌'],
 ['哀感', '悲悲寂悲'],
 ['愛敬', '親親好好親']]

In [9]:
j=0
for d in row_use:
    if d[0] in new_voc:
        continue
    else:
        new_voc.append(d[0])
print(len(new_voc))
#語彙のリストに含まれない感性語辞書の単語を語彙のリストに追加

21192


In [10]:
# ボキャブラリと分かち書き文章から、データセットを作成。
word_to_id = dict((c,i) for i,c in enumerate(new_voc))
id_to_word = dict((i,c) for i,c in enumerate(new_voc))
print(word_to_id["する"])
print(id_to_word[0])

0
する


In [11]:
new_wakati_sentence=[]
for word in wakati_sentences:
    if word in new_voc:
        new_wakati_sentence.append(word)
print(len(new_wakati_sentence))
#分かち書きを行った文の単語に対して、語彙に含まれない物を削除

8914226


In [12]:
# 分かち書き文章を単語IDで表現
wakati_ids = []
for word in new_wakati_sentence:
    wakati_ids.append(word_to_id[word])

print(wakati_ids[:5])
print(id_to_word[140], id_to_word[0], id_to_word[974], id_to_word[2042], id_to_word[3851])

[140, 0, 974, 2042, 3851]
意味 する 記号 ラテン語 合


実際に用いる分かち書き文章はnew_wakati_sentence、用いる語彙はnew_voc、感情値はrow_useに保存されている。

In [13]:
%store wakati_ids
%store id_to_word

Stored 'wakati_ids' (list)
Stored 'id_to_word' (dict)


In [ ]:
%store -r

In [ ]:
#https://github.com/oreilly-japan/deep-learning-from-scratch-2/blob/master/ch03/train.py
#ここから CBOW 準備

import sys
sys.path.append('..')  # 親ディレクトリのファイルをインポートするための設定
from common.trainer import Trainer
from common.optimizer import Adam
from simple_cbow import SimpleCBOW
from common.util import preprocess, create_contexts_target, convert_one_hot, cos_similarity

window_size = 3
hidden_size = 10 # 密ベクトルのサイズ
batch_size = 100 # 一度に処理するサンプル数
max_epoch = 1000 # 重み更新回数（学習回数）

# データセットの準備
vocab_size = len(new_voc)

# ウィンドウサイズで指定された文脈と、その文脈下における対象語を収集
contexts, target = create_contexts_target(wakati_ids, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

print(target[0])

In [ ]:
# モデルの準備
model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [ ]:
# 学習。必要に応じて学習済みファイルから読み込み。
model_filename = "word2vec_momotaro_model.pkl"
trainer_filename = "word2vec_momotaro_trainer.pkl"
answer = "n"
if os.path.exists(filename):
    print("use pretrained file ({})? [y/n]".format(model_filename))
    answer = input()
if answer == "n":
    trainer.fit(contexts, target, max_epoch, batch_size)
    trainer.plot()
    pickle.dump(model, open(model_filename, 'wb'))
    pickle.dump(trainer, open(trainer_filename, 'wb'))
elif answer == "y":
    model = pickle.load(open(model_filename, 'rb'))
    trainer = pickle.load(open(trainer_filename, 'rb'))
    trainer.plot()

In [ ]:
# 学習した密ベクトルを利用しやすいように整形
embeddings = model.word_vecs
word_to_vec = dict()
for index, word in enumerate(vocab):
    vec = embeddings[index]
    word_to_vec[word] = vec

In [ ]:
def closest(embeddings, word, n):
    """密ベクトル集合から、指定された単語に類似した単語を検索して出力する。

    :param embeddings: 単語の密ベクトルを辞書型で保存したもの。{word:vector}
    :param word: 検索対象語。str型。
    :param n: 類似度が最も高いn件まで出力。
    """
    distances = dict()
    for w in embeddings.keys():
        distances[w] = cos_similarity(embeddings[w],embeddings[word])
    d_sorted = collections.OrderedDict(sorted(distances.items(),key = lambda x:x[1] ,reverse = True))
    s_words = list(d_sorted.keys())
    print("closest({})".format(word))
    print(s_words[:n])

    # 参考用に3件まで類似度を出力
    print(cos_similarity(embeddings[word], embeddings[s_words[0]]), word, s_words[0])
    print(cos_similarity(embeddings[word], embeddings[s_words[1]]), word, s_words[1])
    print(cos_similarity(embeddings[word], embeddings[s_words[2]]), word, s_words[2])
    print("----")

closest(word_to_vec,'桃太郎',10)
closest(word_to_vec,'おじいさん',10)
closest(word_to_vec,'猿',10)
closest(word_to_vec,'鬼',10)

In [ ]:
# t-SNE visualization
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
#上記エラーに対応するための環境変数設定。なお、エラー文全文によるとこの方法は非推奨の模様。
#nokmlをインストールすることで解決できるケースが多いようだけど、試した限りでは途中で関連ライブラリのインストールが止まるため、今回は環境変数設定で対応することに。
#参考：https://stackoverflow.com/questions/53648730/omp-error-15-initializing-libiomp5-dylib-but-found-libiomp5-dylib-already-in

# フォント設定
import matplotlib
import matplotlib.font_manager as font_manager
font_path = '/Library/Fonts/Arial Unicode.ttf'
font_prop = font_manager.FontProperties(fname = font_path)
matplotlib.rcParams['font.family'] = font_prop.get_name()

# TSNE設定
from sklearn.manifold import TSNE
tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
labels = []
tokens = []
for w in word_to_vec.keys():
    # 全ての単語を描画したい場合、ここでの品詞判定をせずに、全てlabels.appendすると良い。
    temp = tagger.parse(w).split()[1]
    pos = temp.split(',')[0]
    if pos == '名詞':
        labels.append(w)
        tokens.append(word_to_vec[w])

# 各種パラメータは適宜ドキュメント参照。ここでは固定したいのでシード値も設定。
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
new_values = tsne_model.fit_transform(tokens)
x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])

plt.figure(figsize=(16, 16))
for i in range(len(x)):
    plt.scatter(x[i], y[i])
    plt.annotate(labels[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()